### 순환 신경망

In [1]:
# 순환신경망(Recurrent Neural Network)은 지금까지 살펴본 네트워크와는 입력을 받아들이는 방식과 처리하는 방식에서 
# 차이가 있다. RNN은 순서가 있는 데이터를 입력으로 받고 같은 네트워크를 이용해서 변화하는 입력에 대한 출력을 얻어낸다.
# 순서가 있는 데이터로는 음악, 자연어, 날씨, 주가등 시간의 흐름에 따라 변화하고 그 변화가 의미를 갖는 데이터를 말한다.
# 이번 장에서는 그중 가장 범용적으로 쓰이는 자연어 처리에 RNN을 사용하는 방법을 알아보기로 한다.

In [2]:
# RNN은 일반적인 딥러닝 네트워크처럼 입력 X를 받아서 출력 Y를 반환하지만 추가로 되먹임 구조를 가진다는 차이가 있다.
# In RNN, output is fed back into input
# 되먹임 구조란 어떤 레이어의 출력을 다시 입력으로 받는 구조를 말한다. RNN의 구조를 풀어보면 입력이 X1,X2,X3으로
# 변할 때 같은 네트워크를 사용해서 출력인 Y1,Y2,Y3를 반환하고 있는데 꼭 염두에 둬야 할 점은 출력값이 다음 입력을 
# 받을 때의 RNN 네트워크에도 동일하게 전달되고 있다는 것이다. 즉 RNN 네트워크는 처음에는 X1을 입력으로 받고 그다음
# 에는 X2와 이전 단계 출력값인 Y1, 그다음에는 X3와 이전 단계 출력값인 Y2를 입력으로 받는다. 이과정에서 RNN 네트워크
# 는 동일하게 사용된다.
# RNN은 입력과 출력의 길이에 제한이 없다는 특징이 있다. 다양한 형태의 RNN으로는 
# ONE TO MANY : Image Captioning,  MANY TO ONE : Sentiment Classification
# MANY TO MANY : Machine Translation, Video Classification(Frame level)

### SimpleRNN 레이어

In [3]:
# RNN은 순서가 있는 데이터, 즉 변화하는 입력을 받기 때문에 각 단계에서 변화하는 입력에 대한 계산의 흐름을 파악하는
# 것이 중요하다. Xt-1, Xt, Xt+1,...등은 SimpleRNN에 들어가는 입력을 나타내고 Ht-1, Ht, Ht+1등은 출력값들이다.
# U는 입력에 곱해지는 가중치, W는 출력에 곱해지는 가중치이다. 

##### 단계 t에서의 SimpleRNN레이어의 계산식 ($tanh$는 실수 입력값을 받아 (-1, 1)사이의 출력값을 반환) 
##### $$H_{t} = tanh(U * X_{t} + W * H_{t-1})$$


In [20]:
# SimpleRNN레이어는 tf.keras에서 다음과 같이 생성할 수 있다.
# units는 SimpleRNN레이어에 존재하는 뉴런의 수, return_sequences는 출력으로 시퀀스 전체를 출력할지 여부
# rnn1 = tf.keras.layers.SimpleRNN(units=1, activation='tanh', return_sequences=True)
# 간단한 예제로 시퀀스를 구성하는 앞쪽 4개의 숫자가 주어졌을 때 그다음에 올 숫자를 예측하는 간단한 "시퀀스 예측 모델"
# 을 SimpleRNN레이어를 이용해 만들어 본다.(입력이 [1,2,3,4]일때, [5]를 예측하는 모델을 만드는 것이 목표이다.)

X, Y = [], []
for i in range(6):
    # [0,1,2,3],[1,2,3,4]같은 정수의 시퀀스를 만든다.
    lst = list(range(i, i+4))
    lst = [c/10 for c in lst]
    
    X.append(lst)
    Y.append(round(lst[-1] + 0.1, 1))

    # 위에서 구한 시퀀스의 숫자들을 각각 10으로 나눈 다음 저장한다.
for i in range(len(X)):
    print(X[i], end=', ')
    print(Y[i])   

[0.0, 0.1, 0.2, 0.3], 0.4
[0.1, 0.2, 0.3, 0.4], 0.5
[0.2, 0.3, 0.4, 0.5], 0.6
[0.3, 0.4, 0.5, 0.6], 0.7
[0.4, 0.5, 0.6, 0.7], 0.8
[0.5, 0.6, 0.7, 0.8], 0.9


In [22]:
import numpy as np

X = np.array(X).reshape(6, 4, 1)
Y = np.array(Y)

for i in range(len(X)):
    print(X[i], Y[i])

[[0. ]
 [0.1]
 [0.2]
 [0.3]] 0.4
[[0.1]
 [0.2]
 [0.3]
 [0.4]] 0.5
[[0.2]
 [0.3]
 [0.4]
 [0.5]] 0.6
[[0.3]
 [0.4]
 [0.5]
 [0.6]] 0.7
[[0.4]
 [0.5]
 [0.6]
 [0.7]] 0.8
[[0.5]
 [0.6]
 [0.7]
 [0.8]] 0.9


In [23]:
X[0].shape

(4, 1)

In [24]:
# SimpleRNN을 이용해서 네트워크를 정의한다. 모델구조는 지금까지 이용했던 Sequential Model이고 출력을 위한 Dense
# 레이어가 추가되어 있다.

import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=10, return_sequences=False, input_shape=[4,1]),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 10)                120       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(X, Y, epochs=100, verbose=0)

In [26]:
print(model.predict(X))

1/1 [==============================] - 0s 134ms/step
[[0.39245608]
 [0.5062356 ]
 [0.6031896 ]
 [0.67993   ]
 [0.7349788 ]
 [0.7682954 ]]


In [27]:
print(model.predict(np.array([[0.2], [0.3], [0.4], [0.5]])))

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_3/simple_rnn_3/transpose' defined at (most recent call last):
    File "c:\Users\rubin\anaconda3\envs\tf_gpu\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\rubin\anaconda3\envs\tf_gpu\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\rubin\anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\rubin\anaconda3\envs\tf_gpu\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\rubin\anaconda3\envs\tf_gpu\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\rubin\anaconda3\envs\tf_gpu\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\rubin\anac

In [6]:
x = [[0,1,2,3],[1,2,3,4]]
y = [c/10 for c in x[0]]
z = map(lambda c : c/10, x[0])

for i in y:
    print(i, end='  ')
print()
    
for i in z:
    print(i, end='  ')
print()

for i in x:
    z = map(lambda c : c/10, i)
    for c in z:
        print(c, end='  ')
    print()   

0.0  0.1  0.2  0.3  
0.0  0.1  0.2  0.3  
0.0  0.1  0.2  0.3  
0.1  0.2  0.3  0.4  
